In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 1000)
import seaborn as sns
import unidecode

In [2]:
#Lendo dados
drogas_entrada = pd.read_csv('dados/GESEG_SBX_DROGA_EXPORT.csv', encoding = 'latin-1', sep = ';', index_col=0)
drogas_entrada.head(10)

,ANO_OCOR,NRO_OCOR,NRO_INT_OBJETO,NRO_INT_TIPO_OBJETO,TIPO_OBJETO,QUANTIDADE,ATRIBUTO
ORGAO_OCOR,,,,,,,
100829,2021,1137,23703381,756,Cocaína,Grama,Unidade de Peso
151321,2020,1129,22817480,5,Maconha,Grama,Unidade de Peso
150808,2020,15392,23048116,5,Maconha,Verde,Cor Predominante
100510,2020,16612,23076594,756,Cocaína,4 pinos de cocaína pesando aproximadamente o t...,Descrição
151008,2019,2462,20497307,1025,Drogas,1,Quantidade de Unidade(s)
151008,2019,2462,20497307,1025,Drogas,Porção(es),Tipo de Embalagem
150808,2019,20501,21804599,5,Maconha,Verde,Cor Predominante
152308,2020,5394,23251372,1025,Drogas,01,Quantidade de Unidade(s)
151707,2019,4116,21639890,5,Maconha,Verde,Cor Predominante


In [3]:
#Pivotando valor "QUANTIDADE" da coluna "ATRIBUTOS" com o mesmo "NRO_INT_OBJETO"
drogas_pivotado = drogas_entrada.pivot(index="NRO_INT_OBJETO", columns="ATRIBUTO", values="QUANTIDADE")
drogas_pivotado.head()

ATRIBUTO,Cor Predominante,Descrição,Peso Total,Peso Unitário,Quantidade de Unidade(s),Tipo de Embalagem,Unidade de Peso
NRO_INT_OBJETO,,,,,,,
11228824,Amarelo,NaN,"0,42",NaN,4,Não soube informar,NaN
19338359,Verde,NaN,"2,20",NaN,1,Tijolo(s),Grama
19344958,Verde,NaN,"14,20",NaN,2,Porção(es),Grama
19352770,Branco,NaN,"0,80",NaN,1,Porção(es),Grama
19352780,Verde,NaN,"1,60",NaN,1,Cigarro(s),Grama


In [4]:
#Agrupando restante dos dados não pivotados pelo mesmo "NRO_INT_OBJETO"
drogas_agrupado = drogas_entrada[['NRO_INT_OBJETO','ANO_OCOR', 'NRO_OCOR','NRO_INT_TIPO_OBJETO','TIPO_OBJETO']].groupby(["NRO_INT_OBJETO"]).min()
drogas_agrupado.head(10)

,ANO_OCOR,NRO_OCOR,NRO_INT_TIPO_OBJETO,TIPO_OBJETO
NRO_INT_OBJETO,,,,
11228824,2009,3209,1025,Drogas
19338359,2018,3315,5,Maconha
19344958,2018,3319,5,Maconha
19352770,2018,3139,756,Cocaína
19352780,2018,3143,5,Maconha
19352781,2018,3143,5,Maconha
19359630,2018,2674,5,Maconha
19359634,2018,2675,5,Maconha
19359637,2018,2676,5,Maconha


In [5]:
#Ordena por "NRO_INT_OBJETO" paraa concatenar de forma simples e rápida
drogas_agrupado_sorted = drogas_agrupado.sort_values(by=['NRO_INT_OBJETO'])
drogas_pivotado_sorted = drogas_pivotado.sort_values(by=['NRO_INT_OBJETO'])
#Garantia de mesmo numero de linhas para concatenação
display(drogas_agrupado_sorted.shape, drogas_pivotado_sorted.shape)
if drogas_agrupado_sorted.shape[0] != drogas_pivotado_sorted.shape[0]:
    display("ATENÇÃO: QTDE de registros diferentes, houve algum problema nos passos anteriores.")

(113231, 4)

(113231, 7)

In [6]:
#Concatenando as duas tabelas e refaz o índice numérico
drogas_df = pd.concat([drogas_agrupado_sorted, drogas_pivotado_sorted],axis=1).reset_index()
#Renomeando Coluans
drogas_df = drogas_df.rename(columns={"NRO_INT_OBJETO": "CHAVE_OBJETO", "TIPO_OBJETO": "TXT_TIPO_OBJETO"})
#Mostrando o resultado com descrição não nula
drogas_df[drogas_df['Descrição'].notnull()].head(5)

,CHAVE_OBJETO,ANO_OCOR,NRO_OCOR,NRO_INT_TIPO_OBJETO,TXT_TIPO_OBJETO,Cor Predominante,Descrição,Peso Total,Peso Unitário,Quantidade de Unidade(s),Tipo de Embalagem,Unidade de Peso
216,19903084,2018,4727,5,Maconha,Verde,"maconha prensada embalada em plastico, 5 gr","5,00","5,00",1,Pacote(s),Grama
217,19903086,2018,4727,5,Maconha,Verde,"maconha prensada embalada em plastico, 1,4 gr","1,40","1,40",1,Pacote(s),Grama
218,19903569,2018,6732,5,Maconha,Verde,Um porção de erva esverdeada semelhante a maco...,"2,40","2,40",1,Porção(es),Grama
219,19903819,2018,30412,5,Maconha,Verde,DOIS GRAMAS DE SUBSTANCIA ESVERDEADA DE ASPECT...,"2,00",NaN,3,Porção(es),Grama
220,19903932,2018,8070,1025,Drogas,Verde,uma pequena porção de erva de cor esverdeada s...,NaN,NaN,1,Porção(es),Miligrama


# Reclassificação do TIPO DE OBJETO

In [7]:
#Confere frequencia de cada categoria
freq_tipo = drogas_df.groupby(["TXT_TIPO_OBJETO"]).size().reset_index(name="Quantidade").set_index("TXT_TIPO_OBJETO").sort_index()
freq_tipo['Frequência'] = drogas_df['TXT_TIPO_OBJETO'].value_counts(normalize=True).sort_index().round(6).astype(str)
freq_tipo.T


TXT_TIPO_OBJETO,Cocaína,Crack,Drogas,Maconha
Quantidade,20063,14476,37528,41164
Frequência,0.177186,0.127845,0.331429,0.36354


In [8]:
#cria um campo descricao em unicode e minusculo
def padronizada(txt):
    if pd.isna(txt):
        return pd.NA
    return unidecode.unidecode(str(txt).replace(",","").replace(".","")).lower()
   
drogas_df['DESC_PADRAO'] = drogas_df['Descrição'].apply(padronizada)

#Monta serie de drogas para reclassificação
DrogasReclassificar = drogas_df.loc[drogas_df['TXT_TIPO_OBJETO'] == 'Drogas'].reset_index(drop=True)
#Salvando drogas ja já classificadas
JaClassificadas = drogas_df.loc[drogas_df['TXT_TIPO_OBJETO'] != 'Drogas'].reset_index(drop= True)


In [9]:
#Tratando ajustes de drogas
def reclassificar(descricao):
    if pd.isna(descricao):
        return 'Drogas'
    if any(item in descricao for item in ['maconha','canabis','cannabis']):
        #cigarro
        return 'Maconha'
    elif any(item in descricao for item in ['crack','crak']):
        # pedra
        return 'Crack'
    elif any(item in descricao for item in ['cocaina','cacaina']):
        # pinos, po
        return 'Cocaína'
    else:
        return 'Drogas'

DrogasReclassificar['RECLASSIFICAÇÃO'] = DrogasReclassificar['DESC_PADRAO'].apply(reclassificar)

In [10]:
#Junta novamente com as drogas já classificadas e seta a reclassificacao para o dado anterior
reclassificacao = pd.concat([DrogasReclassificar, JaClassificadas])
reclassificacao['RECLASSIFICAÇÃO'] = reclassificacao['RECLASSIFICAÇÃO'].fillna(value=reclassificacao.TXT_TIPO_OBJETO)

In [11]:
freq_reclassificada = reclassificacao.groupby(['RECLASSIFICAÇÃO']).size().reset_index(name="Quantidade").set_index('RECLASSIFICAÇÃO').sort_index()
freq_reclassificada['Frequência'] = reclassificacao['RECLASSIFICAÇÃO'].value_counts(normalize=True).sort_index().round(6).astype(str)
display(freq_tipo.T, "Total de registros=" + str(freq_tipo["Quantidade"].sum()))
display(freq_reclassificada.T, "Total de registros=" + str(freq_reclassificada["Quantidade"].sum()))

TXT_TIPO_OBJETO,Cocaína,Crack,Drogas,Maconha
Quantidade,20063,14476,37528,41164
Frequência,0.177186,0.127845,0.331429,0.36354


'Total de registros=113231'

RECLASSIFICAÇÃO,Cocaína,Crack,Drogas,Maconha
Quantidade,28080,20349,5928,58874
Frequência,0.247989,0.179712,0.052353,0.519946


'Total de registros=113231'

In [12]:
def plota_grafico(df, campo_x):
    palette ={"Maconha": "green", "Cocaína": "white", "Crack": "yellow", "Drogas": "blue"}
    f, (ax3) = plt.subplots(1, 1, figsize=(14, 5))
    sns.set(font_scale=1.5)
    sns.countplot(x=campo_x, data=df, ax=ax3, palette= palette, alpha = 0.7, linewidth=2.5, edgecolor=".0")
    ax3.set_facecolor('#ababab')
    ax3.patch.set_alpha(0.3)
    ax3.set_xlabel("Tipo de Objeto",fontsize=20)
    ax3.set_ylabel("Quantidade",fontsize=20)
    ax3.set_xticklabels(ax3.get_xmajorticklabels(), fontsize = 15)
    plt.grid(axis='x')
    plt.tight_layout()

In [13]:
plota_grafico(drogas_df, "TXT_TIPO_OBJETO")

In [14]:
#Plotando Reclassificado
plota_grafico(reclassificacao, "RECLASSIFICAÇÃO")

In [15]:
#Algumas análises textuais
#reclassificacao.loc[(reclassificacao['RECLASSIFICAÇÃO'] == 'Drogas') & (reclassificacao['DESC_PADRAO'].notna()) & (reclassificacao['DESC_PADRAO'].str.contains("cigarro"))].head(10)['DESC_PADRAO']

reclassificacao.loc[(reclassificacao['RECLASSIFICAÇÃO'] == 'Drogas')
    & (reclassificacao['DESC_PADRAO'].notna())
    & ((reclassificacao['DESC_PADRAO'].str.contains("pino"))
        | (reclassificacao['Descrição'].str.contains("pó")))].head(10)['DESC_PADRAO']


55                 aprox nove gramas na embalagem (pinos)
92                          separadas em pinos para venda
252                                    po branco 3 pinos 
318     um pacote contendo diversos pinos (embalagens ...
484             um pino com im pouco de substancia branca
744            1-um pino que pesou 1-um grama c/embalagem
1122    seis porcoes de po branco embalados individual...
1275       50 pinos transparentes peso total com os pinos
1724             a droga se encontra em pinos de plastico
1835                      pinos pesando 13g com embalagem
Name: DESC_PADRAO, dtype: object

In [16]:
reclassificacao.columns

Index(['CHAVE_OBJETO', 'ANO_OCOR', 'NRO_OCOR', 'NRO_INT_TIPO_OBJETO',
       'TXT_TIPO_OBJETO', 'Cor Predominante', 'Descrição', 'Peso Total',
       'Peso Unitário', 'Quantidade de Unidade(s)', 'Tipo de Embalagem',
       'Unidade de Peso', 'DESC_PADRAO', 'RECLASSIFICAÇÃO'],
      dtype='object')

# Exportação

In [17]:
reclassificacao = reclassificacao.set_index("CHAVE_OBJETO")
reclassificacao.to_csv('dados/resultado_drogas.csv', encoding = 'latin-1', sep = ';', header=True, quoting=csv.QUOTE_NONNUMERIC)